In [1]:
using JuMP
using GLPKMathProgInterface

In [2]:
#instância de exemplo
n = 5
g = 3

pv = [1, 2, 3, 4, 5]
vx = [50, 0, 23, 55, 103]
vy = [-25, 0, 27, 28, 2]
M = [5, 4, 6]

da = Array{Float64,2}(n,n);
for i = 1:n
    for j = (i+1):n
        dx = vx[i] - vx[j]
        dy = vy[i] - vy[j]
        da[i, j] = sqrt(dx*dx + dy*dy)
        da[j, i] = sqrt(dx*dx + dy*dy)
    end
end

#"infinito"
H = 100

100

In [3]:
#modelo e variaveis do problema
m = Model(solver = GLPKSolverMIP())

alpha = 0.05

@variable(m, minD[1:g] >= 0)
@variable(m, x[1:n, 1:g], Bin)


5×3 Array{JuMP.Variable,2}:
 x[1,1]  x[1,2]  x[1,3]
 x[2,1]  x[2,2]  x[2,3]
 x[3,1]  x[3,2]  x[3,3]
 x[4,1]  x[4,2]  x[4,3]
 x[5,1]  x[5,2]  x[5,3]

In [4]:
@objective(m, Max, sum(minD[i] for i in 1:g))


minD[1] + minD[2] + minD[3]

In [5]:
for v = 1:n
	for u = v+1:n
        	for i = 1:g
            	@constraint(m, minD[i] <= da[u,v] + (2-(x[v,i]+x[u,i]))*H)
	        end
	end

	@constraint(m, sum(x[v,k] for k in 1:g) == 1)
end

for k = 1:g
	@constraints(m, begin
		(1-alpha)*M[k] <= sum(x[v,k]*pv[v] for v in 1:n)
        sum(x[v,k]*pv[v] for v in 1:n) <= (1+alpha)*M[k]
		end)
end



In [6]:
print(m)


Max minD[1] + minD[2] + minD[3]
Subject to
 minD[1] + 100 x[1,1] + 100 x[2,1] ≤ 255.90169943749476
 minD[2] + 100 x[1,2] + 100 x[2,2] ≤ 255.90169943749476
 minD[3] + 100 x[1,3] + 100 x[2,3] ≤ 255.90169943749476
 minD[1] + 100 x[1,1] + 100 x[3,1] ≤ 258.591808301161
 minD[2] + 100 x[1,2] + 100 x[3,2] ≤ 258.591808301161
 minD[3] + 100 x[1,3] + 100 x[3,3] ≤ 258.591808301161
 minD[1] + 100 x[1,1] + 100 x[4,1] ≤ 253.23532661682466
 minD[2] + 100 x[1,2] + 100 x[4,2] ≤ 253.23532661682466
 minD[3] + 100 x[1,3] + 100 x[4,3] ≤ 253.23532661682466
 minD[1] + 100 x[1,1] + 100 x[5,1] ≤ 259.4810894318522
 minD[2] + 100 x[1,2] + 100 x[5,2] ≤ 259.4810894318522
 minD[3] + 100 x[1,3] + 100 x[5,3] ≤ 259.4810894318522
 x[1,1] + x[1,2] + x[1,3] = 1
 minD[1] + 100 x[2,1] + 100 x[3,1] ≤ 235.4682957019364
 minD[2] + 100 x[2,2] + 100 x[3,2] ≤ 235.4682957019364
 minD[3] + 100 x[2,3] + 100 x[3,3] ≤ 235.4682957019364
 minD[1] + 100 x[2,1] + 100 x[4,1] ≤ 261.7170964968379
 minD[2] + 100 x[2,2] + 100 x[4,2] ≤ 261.717

In [7]:
solve(m)

:Optimal

In [10]:
for i = 1:g
    println("peso alvo do grupo $(i) = $(M[i])")
    println("peso total dos vertices do grupo $(i) = $(sum(getvalue(x[v,i])*pv[v] for v in 1:n))")
    println("distância minima dos vertices do grupo $(i) = $(getvalue(minD[i]))")
    println("vertices: $(getvalue(x[1:n, i]))")
end

peso alvo do grupo 1 = 5)
peso total dos vertices do grupo 1 = 5.0
distância minima dos vertices do grupo 1 = 154.58937625582473
vertices: [0.0, 0.0, 0.0, 0.0, 1.0]
peso alvo do grupo 2 = 4)
peso total dos vertices do grupo 2 = 4.0
distância minima dos vertices do grupo 2 = 132.01562118716424
vertices: [0.0, 0.0, 0.0, 1.0, 0.0]
peso alvo do grupo 3 = 6)
peso total dos vertices do grupo 3 = 6.0
distância minima dos vertices do grupo 3 = 35.4682957019364
vertices: [1.0, 1.0, 1.0, 0.0, 0.0]
